In [57]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

/Users/hsh/dev_project/sb-fsts


In [58]:
from app.utils.crud_sql import SQLExecutor
from app.utils.database import get_db, get_db_session

sql_executor = SQLExecutor()

In [4]:
query = """
        SELECT 
            *
        FROM 
            fsts.test_table
    """

params = {}

with get_db_session() as db:
    result = sql_executor.execute_select(db, query, params)

print(result)

[{'id': 'id3', 'name': 'name3'}, {'id': 'id5', 'name': 'name5'}, {'id': 'id6', 'name': 'name6'}, {'id': 'id7', 'name': 'name7'}, {'id': 'id1', 'name': 'updated_name1'}, {'id': 'id2', 'name': 'upserted_name2'}, {'id': 'id4', 'name': 'upserted_name4'}]


In [ ]:
# RETURNING * 꼭 붙여야 함.
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

id = 'id4'
name = 'name4'

params = {
    "id": id,
    "name": name
}

with get_db_session() as db:
    result = sql_executor.execute_insert(db, query, params)

In [ ]:
#여러 데이터를 한번에 삽입하고 싶을 때
# 여러 데이터 정의
data = [
    {"id": "id5", "name": "name5"},
    {"id": "id6", "name": "name6"},
    {"id": "id7", "name": "name7"}
]

# 다중 데이터 삽입 쿼리
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

# 데이터 삽입 및 결과 저장
results = []
with get_db_session() as db:
    for row in data:
        result = sql_executor.execute_insert(db, query, row)
        results.append(result)

print("삽입된 데이터:")
print(results)

In [6]:
# UPDATE 테스트
query_update = """
    UPDATE fsts.test_table
    SET name = :name
    WHERE id = :id
    RETURNING *;
"""
#:는 파라미터 바인딩 형식

params_update = {
    "id": "id2",
    "name": "HONG"
}

with get_db_session() as db:
    result_update = sql_executor.execute_update(db, query_update, params_update)

Update succeeded: [{'id': 'id2', 'name': 'HONG'}]


# DynamoDB

In [22]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, ListAttribute, MapAttribute
from datetime import datetime
import time

# 매매 이력 (trading_history)에서 사용될 속성 정의
class TradeHistory(MapAttribute):
    trading_logic = UnicodeAttribute()
    symbol = UnicodeAttribute()
    symbol_name = UnicodeAttribute()
    position = UnicodeAttribute()
    price = NumberAttribute()
    quantity = NumberAttribute()
    created_at = NumberAttribute()
    updated_at = UnicodeAttribute(null=True)
    data_type = UnicodeAttribute()

# DynamoDB 테이블 모델 정의
class TradingBot(Model):
    class Meta:
        table_name = "fsts-trading-bot"
        region = "ap-northeast-2"

    trading_bot_name = UnicodeAttribute(hash_key=True)  # PK 설정
    created_at = NumberAttribute()
    trading_history = ListAttribute(of=TradeHistory)  # 리스트 저장 가능

# 테이블 생성 (최초 1회 실행)
# if not TradingBot.exists():
#     TradingBot.create_table(read_capacity_units=5, write_capacity_units=5, wait=True)

In [45]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute
import time
import uuid

class TradingHistory(Model):
    class Meta:
        table_name = "fsts-trading-history"
        region = "ap-northeast-2"

    trading_bot_name = UnicodeAttribute(hash_key=True)  # ✅ PK
    created_at = NumberAttribute(range_key=True)  # ✅ SK (밀리세컨드 단위)
    updated_at = UnicodeAttribute(null=True)
    symbol = UnicodeAttribute()
    position = UnicodeAttribute()
    price = NumberAttribute()
    quantity = NumberAttribute()
    data_type = UnicodeAttribute()

In [40]:
from pynamodb.transactions import TransactWrite
from pynamodb.connection import Connection
from botocore.exceptions import ClientError

def add_trade(trading_bot_name, symbol, position, price, quantity, data_type):
    max_retries = 3  # 최대 재시도 횟수
    retry_count = 0

    while retry_count < max_retries:
        try:
            created_at = int(time.time() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

            new_trade = TradingHistory(
                trading_bot_name=trading_bot_name,
                created_at=created_at,
                updated_at=None,
                symbol=symbol,
                position=position,
                price=price,
                quantity=quantity,
                data_type=data_type
            )

            connection = Connection()

            with TransactWrite(connection=connection) as transaction:
                transaction.save(new_trade, condition=(TradingHistory.created_at.does_not_exist()))
                print(f"✅ 트랜잭션 성공: {created_at}")
                return True  # 성공적으로 저장되면 종료

        except ClientError as e:
            if e.response["Error"]["Code"] == "TransactionCanceledException":
                print("❌ 중복된 created_at 감지! 새로운 값으로 재시도...")
                retry_count += 1
            else:
                raise  # 다른 에러 발생 시 예외 던지기

    print("🚨 최대 재시도 횟수 초과! 거래 저장 실패")
    return False

In [66]:
def execute_save(data_model):
        max_retries = 3  # 최대 재시도 횟수
        retry_count = 0

        while retry_count < max_retries:
            try:
                connection = Connection()

                created_at = int(time.time() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

                with TransactWrite(connection=connection) as transaction:
                    # transaction.save(data_model, condition=(TradingHistory.created_at.does_not_exist()))
                    result = transaction.save(data_model, condition=(type(data_model).created_at.does_not_exist()))
                    print(f"✅ 트랜잭션 성공: {created_at}, {result}")
                    return True  # 성공적으로 저장되면 종료

            except ClientError as e:
                if e.response["Error"]["Code"] == "TransactionCanceledException":
                    print("❌ 중복된 created_at 감지! 새로운 값으로 재시도...")
                    retry_count += 1
                else:
                    raise  # 다른 에러 발생 시 예외 던지기

        print("🚨 최대 재시도 횟수 초과! 거래 저장 실패")
        return False

In [42]:
add_trade(
    trading_bot_name="MyTradingBot_1234",
    symbol="AAPL",
    position="BUY",
    price=200,
    quantity=1,
    data_type="test"
)

✅ 트랜잭션 성공: 1740408172439


True

In [71]:
kospi = [
  {"name": "삼성전자", "symbol": "005930"},
  {"name": "SK하이닉스", "symbol": "000660"},
  {"name": "LG에너지솔루션", "symbol": "373220"},
  {"name": "삼성바이오로직스", "symbol": "207940"},
  {"name": "현대차", "symbol": "005380"},
  {"name": "셀트리온", "symbol": "068270"},
  {"name": "기아", "symbol": "000270"},
  {"name": "NAVER", "symbol": "035420"},
  {"name": "한화에어로스페이스", "symbol": "012450"},
  {"name": "KB금융", "symbol": "105560"},
  {"name": "HD현대중공업", "symbol": "329180"},
  {"name": "POSCO홀딩스", "symbol": "005490"},
  {"name": "한화오션", "symbol": "042660"},
  {"name": "현대모비스", "symbol": "012330"},
  {"name": "메리츠금융지주", "symbol": "138040"},
  {"name": "신한지주", "symbol": "055550"},
  {"name": "삼성물산", "symbol": "028260"},
  {"name": "카카오", "symbol": "035720"},
  {"name": "SK이노베이션", "symbol": "096770"},
  {"name": "HMM", "symbol": "011200"},
  {"name": "LG화학", "symbol": "051910"},
  {"name": "삼성화재", "symbol": "000810"},
  {"name": "두산에너빌리티", "symbol": "034020"},
  {"name": "삼성생명", "symbol": "032830"},
  {"name": "크래프톤", "symbol": "259960"},
  {"name": "하나금융지주", "symbol": "086790"},
  {"name": "HD한국조선해양", "symbol": "009540"},
  {"name": "삼성SDI", "symbol": "006400"},
  {"name": "고려아연", "symbol": "010130"},
  {"name": "한국전력", "symbol": "015760"},
  {"name": "삼성중공업", "symbol": "010140"},
  {"name": "LG전자", "symbol": "066570"},
  {"name": "기업은행", "symbol": "024110"},
  {"name": "KT", "symbol": "030200"},
  {"name": "HD현대일렉트릭", "symbol": "267260"},
  {"name": "KT&G", "symbol": "033780"},
  {"name": "SK스퀘어", "symbol": "402340"},
  {"name": "SK텔레콤", "symbol": "017670"},
  {"name": "우리금융지주", "symbol": "316140"},
  {"name": "카카오뱅크", "symbol": "323410"},
  {"name": "포스코퓨처엠", "symbol": "003670"},
  {"name": "한화생명", "symbol": "088350"},
  {"name": "CJ대한통운", "symbol": "000120"},
  {"name": "팬오션", "symbol": "028670"},
  {"name": "현대엘리베이", "symbol": "017800"},
  {"name": "코스맥스", "symbol": "192820"},
  {"name": "영원무역", "symbol": "111770"},
  {"name": "현대해상", "symbol": "001450"},
  {"name": "제일기획", "symbol": "030000"},
  {"name": "HL만도", "symbol": "204320"},
  {"name": "SK아이이테크놀로지", "symbol": "361610"},
  {"name": "한전KPS", "symbol": "051600"},
  {"name": "롯데쇼핑", "symbol": "023530"},
  {"name": "한미사이언스", "symbol": "008930"},
  {"name": "한올바이오파마", "symbol": "009420"},
  {"name": "HD현대인프라코어", "symbol": "042670"},
  {"name": "풍산", "symbol": "103140"},
  {"name": "BGF리테일", "symbol": "282330"},
  {"name": "아모레G", "symbol": "002790"},
  {"name": "DL이앤씨", "symbol": "375500"},
  {"name": "오뚜기", "symbol": "007310"},
  {"name": "OCI홀딩스", "symbol": "010060"},
  {"name": "코스모신소재", "symbol": "005070"},
  {"name": "신세계", "symbol": "004170"},
  {"name": "GS건설", "symbol": "006360"},
  {"name": "호텔신라", "symbol": "008770"},
  {"name": "이수스페셜티케미컬", "symbol": "457190"},
  {"name": "씨에스윈드", "symbol": "112610"},
  {"name": "한국앤컴퍼니", "symbol": "000240"},
  {"name": "녹십자", "symbol": "006280"},
  {"name": "DGB금융지주", "symbol": "139130"},
  {"name": "에스엘", "symbol": "005850"},
  {"name": "한국콜마", "symbol": "161890"},
  {"name": "대우건설", "symbol": "047040"},
  {"name": "대웅제약", "symbol": "069620"},
  {"name": "금호타이어", "symbol": "073240"},
  {"name": "한솔케미칼", "symbol": "014680"},
  {"name": "현대백화점", "symbol": "069960"},
  {"name": "하이트진로", "symbol": "000080"},
  {"name": "에스디바이오센서", "symbol": "137310"},
  {"name": "GS리테일", "symbol": "007070"},
  {"name": "영원무역홀딩스", "symbol": "009970"}
]

In [74]:
from app.utils.dynamodb.model.stock_symbol_model import StockSymbol
from pytz import timezone


kst = timezone("Asia/Seoul")
# 현재 시간을 KST로 변환

for i in kospi:
    current_time = datetime.now(kst)
    created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

    model = StockSymbol(
        symbol=i['symbol'],
        created_at=created_at,
        updated_at=None,
        symbol_name=i['name'],
        type='kospi200'
    )
    execute_save(model)

✅ 트랜잭션 성공: 1741498228029, None


TransactWriteError: Failed to write transaction items

In [103]:
from app.utils.dynamodb.model.user_info_model import UserInfo

current_time = datetime.now(kst)
created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

model = UserInfo(
    name='홍석형',
    created_at=created_at,
    updated_at=None,
    id='id1',
    password='password1',
    app_key='PSyTGF07QupJyV76XGm3mkgcr4RDvSeODpVZ',
    secret_key='eteoHNN+iHktbHC1TOKNdDc2ecFHqwyA+o1OijESqRtWY2cirhUqbiuFfO5zmEPNqB8/P0RSBuTjZnPq4zc5u3dKHIg/HOFQqmZcCik621aWqti5MBReqNpr/NChcs8edoBKd4cgJaC47m3IKncU4GglKzWNqHtic/4X8lmOAZx0oDGuFkI=',
    kis_id='bnuazz15',
    account='67737279'
)
execute_save(model)

✅ 트랜잭션 성공: 1741499975380, None


TransactWriteError: Failed to write transaction items

### 데이터 조회

In [53]:
# 특정 trading_bot_name의 데이터 조회
history = TradingHistory.scan()

for trade in history:
    print(f"- Symbol: {trade.symbol}, Price: {trade.price}, Quantity: {trade.quantity}, Created At: {trade.created_at}")

- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740407407897
- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740408172439


In [52]:
# 특정 trading_bot_name의 데이터 조회
history = TradingHistory.query("MyTradingBot_1234")

for trade in history:
    print(f"- Symbol: {trade.symbol}, Price: {trade.price}, Quantity: {trade.quantity}, Created At: {trade.created_at}")

- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740407407897
- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740408172439


In [99]:
kospi200_items = StockSymbol.scan(
    filter_condition=(StockSymbol.type == 'kospi200')
)

In [101]:
for i in kospi200_items:
    print(i.symbol)

012330
111770
204320
007310
008770
032830
086790
017800
047040
006360
005380
005930
023530
009420
001450
034020
207940
028260
009540
028670
015760
030000
010140
402340
012450
030200
068270
017670
007070
008930
055550
138040
000270
014680
042670
361610
000080
112610
096770
042660
139130
267260
069960
000120
323410
006400
373220
137310
051910
192820
004170
009970
088350
033780
005490
010060
011200
457190
282330
103140
000240
105560
316140
003670
375500
005070
329180
002790
000660
010130
161890
005850
035420
259960
024110
069620
066570
006280
035720
051600
000810
